In [1]:
import sys
sys.path.append("../../FinNLP")

In [2]:
import os
import requests
import shutil
import pandas as pd
from finnlp.data_engineering.data_cleaning import * 

# Downloading sample data

In [3]:
def download_parquet_files(url_list, local_dir):
    for url in url_list:
        file_name = url.split('/')[-1]
        local_file = os.path.join(local_dir, file_name)
        if not os.path.exists(local_dir):
            os.makedirs(local_dir)

        r = requests.get(url, stream=True)
        if r.status_code == 200:
            with open(local_file, 'wb+') as f:
                r.raw.decode_content = True
                shutil.copyfileobj(r.raw, f)
        else:
            print('download failed: ', url)

def web_data_prepare(name):
    r = requests.get("https://datasets-server.huggingface.co/parquet?dataset="+name)
    j = r.json()
    urls = [f['url'] for f in j['parquet_files'] if f['split'] == 'train']
    train_urls = [f['url'] for f in j['parquet_files'] if f['config'] == name and f['split'] == 'train']
    test_urls = [f['url'] for f in j['parquet_files'] if f['config'] == name and f['split'] == 'validation']
    download_parquet_files(train_urls, 'train_dataset')
    download_parquet_files(test_urls, 'test_dataset')

    train_dataset = pd.read_parquet('./train_dataset', engine='pyarrow')
    test_dataset = pd.read_parquet('./test_dataset', engine='pyarrow')

    # train_dataset.rebalance()
    # test_dataset.rebalance()
    return train_dataset, test_dataset



In [4]:
name = 'blog_authorship_corpus'
sample_ratio = 0.01
# train_dataset, test_dataset=web_data_prepare(name = name)
train_dataset = pd.read_parquet('./train_dataset', engine='pyarrow')
test_dataset = pd.read_parquet('./test_dataset', engine='pyarrow')
train_dataset=train_dataset.sample(frac=sample_ratio)
test_dataset=test_dataset.sample(frac=sample_ratio)
train_dataset

,text,date,gender,age,horoscope,job
395587,it has just hit me that in a matter of a few s...,"04,August,2004",female,27,Virgo,indUnk
275435,"Ok, Dear Dopugie/ Ben/ Matt/ anyone who can he...","27,June,2004",male,14,Sagittarius,Student
634637,"ooyeah/Happy Roctober = Miriam? Anyway, just ...","03,August,2004",female,24,Libra,Arts
264675,Election season is in the air! And I know that...,"19,February,2004",male,24,Gemini,indUnk
628907,His face bore the the signs of the many battle...,"03,February,2004",male,24,Libra,indUnk
...,...,...,...,...,...,...
41392,Everything You Wanted to Know About Oscillatin...,"04,October,2003",female,40,Gemini,Law
71956,Sick Chickens March has not been a good mont...,"08,August,2004",male,26,Capricorn,Communications-Media
21415,Gunshot as you are more likely to die quickly ...,"26,March,2003",male,17,Taurus,Technology
255686,Well I am sad to report that my Uncle has pass...,"07,March,2004",female,27,Capricorn,indUnk


# Junk Data
Large-scale datasets often contain an uneven distribution of text representation, which includes a significant amount of nonsensical and boilerplate text - such as HTML tags.

The presence of such "noise" or irrelevant content in the dataset is detrimental to the training of predictive models, specifically those that operate by predicting the next token based on all previous ones. Therefore, it's crucial to clean the dataset and remove these undesired elements prior to the training phase.

This piece of Python code calculated a measure of "impurity" in text documents, and then computing the proportion of documents that exceed a certain impurity threshold. It defines a compiled regular expression that matches any of the following suspicious characters: &, #, <, >, {, }, [, ].

In [5]:
re_e = r'[&#<>{}\[\]\\]'
threshold = 0.01
df, impurity_ratio = junk_eliminate(train_dataset, re_e, threshold)
impurity_ratio

0.049724557842853

# Biased Content
It is crucial in the training of language models to be vigilant and potentially apply tools to exclude toxic content from the pre-training datasets. This practice helps to prevent the models from demonstrating bias or generating detrimental content in subsequent applications.

One approach to address this issue is by scanning the text for offensive words. For instance, the creators of the C4 dataset have implemented such a filtering mechanism. The follow code references this word list that they open source.

The following code utilizes the word list to quantify the "biased content ratio" in the dataset.

In [6]:
non_toxic_df, biased_content_ratio = toxic_eliminate(df = train_dataset, l_kind = 'en')
biased_content_ratio

0.1284430269643375

# Too Short Document
The aim of language modeling is to master the generation of text based on preceding tokens. In this scenario, eliminating extremely brief documents (text consisting of fewer than approximately 100 tokens) from the corpus could aid in the reduction of noise, by producing contiguous text to model dependencies within the text.

Use the Hugging Face Transformers library to tokenize text and then calculate the proportion of documents that are "too short" in a dataset. This example converts text into tokens that the BERT model can understand. Choose a tokenizer for your model.

In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
df = train_dataset
not_short_df, too_short_doc_ratio = short_eliminate(df, tokenizer, min_len=100)
too_short_doc_ratio

0.40895911858509715

# Contamination
Typically, ensuring the segregation of training and testing data is rather straightforward in machine learning. However, things become complicated in the context of large language models where both the training and benchmarking datasets are collected from the internet.

For instance, the performance evaluation of a large language model using benchmark data (like question-answer pairs) can be significantly affected if the benchmark data also features in the model's training set. The procedure of eliminating instances from the training datasets that intersect with the existing benchmarking datasets is called "decontamination".

This Python code below is being used to quantify the contamination problem lying in the datasets, i.e., the proportion of documents in the test set that also appear in the training set using N-grams.

The approach here is from GPT-3 paper. OpenAI defined a test document as contaminated if any N-gram overlap existed with any training document. (They used a range of N values between 8 and 13 depending on dataset.) When constructing the WebText dataset, OpenAI researchers decontaminated the data by eliminating all Wikipedia content from the training set. This was necessary as Wikipedia data was heavily used in their benchmark datasets.

In [8]:
contamination_ratio = contamination_eliminate(train_dataset, test_dataset)
contamination_ratio

0.021108179419525065

# Duplication
When datasets are created by scraping raw text from the Internet, this will often result in the same sequences being repeated multiple times. This paper mentions a single 50 word sequence that is repeated in the C4 dataset 60,000 times.

Deduplication helps prevent models from outputting verbatim training data when there are many duplicates, and makes models less vulnerable to privacy attacks. Deduplication can also improve model training efficiency and prevent benchmark contamination.

Tools & Tutorials
The GPT-3 paper mentions they fuzzily deduplicated documents within each dataset using Spark’s MinHashLSH implementation with 10 hashes.

deduplicate-text-datasets is an ExactSubstr deduplication implementation (written in Rust) along with the scripts to perform ExactSubstr deduplication and inspect the results (written in Python).

datasketch gives you probabilistic data structures that can process and search very large amount of data super fast, with little loss of accuracy.

This article provides a MinHash walkthrough to demonstrate how to implement a parallelel deduplication.

The following code uses the datasketch library and LSH (Locality Sensitive Hashing) to deduplicate the dataset. For each text in the DataFrame, it creates a query MinHash object and performs a query on the LSH index to find similar documents.

It worths to mention that the de-duplication process usually requires a lot of computational resources (CPU and RAM) due to the size of web crawl datasets and it's therefore recommended to run such computations in distributed settings.

In [9]:
unique_documents, duplication_ratio = duplication_eliminate(train_dataset.sample(frac=sample_ratio))
duplication_ratio

0.0

Thanks to [HuggingFace-Datasets-Text-Quality-Analysis](https://huggingface.co/spaces/Dreamsome/HuggingFace-Datasets-Text-Quality-Analysis)